# Clustering Crypto

In [141]:
# Initial imports
import pandas as pd
import hvplot.pandas
import plotly.express as px
#from path import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [142]:
# Load the crypto_data.csv dataset.
file_path = "Resources/crypto_data.csv"

crypto_df = pd.read_csv(file_path)

crypto_df.head(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015,2015 coin,X11,True,PoW/PoS,NaN,0
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [143]:
crypto_df.shape

(1252, 7)

In [144]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df.drop(crypto_df[crypto_df['IsTrading'] == False].index)

In [145]:
#crypto_df = crypto_df.drop(columns="Unnamed: 0")
crypto_df = crypto_df.set_index('Unnamed: 0')

crypto_df.index.name = None

In [146]:
crypto_df.shape

(1144, 6)

In [147]:
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [148]:
crypto_df.value_counts('Algorithm')

Algorithm
Scrypt                394
X11                   182
SHA-256               121
X13                    54
PoS                    42
                     ... 
Proof-of-Authority      1
Proof-of-BibleHash      1
QUAIT                   1
QuarkTX                 1
vDPOS                   1
Length: 89, dtype: int64

In [149]:
crypto_df.count().isnull()

CoinName           False
Algorithm          False
IsTrading          False
ProofType          False
TotalCoinsMined    False
TotalCoinSupply    False
dtype: bool

In [150]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns="IsTrading")

In [151]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()

In [152]:
crypto_df.sort_values(by=["TotalCoinsMined"])

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
FIII,Fiii,SHA3-256,DPoC,-5.917978e+09,5000000000
LBTC,LiteBitcoin,Scrypt,PoW,0.000000e+00,1000000000
RIPO,RipOffCoin,Scrypt,PoW,0.000000e+00,12000000
BASH,LuckChain,Scrypt,PoW/PoS,0.000000e+00,1000000000
CSH,CashOut,SHA-256,PoW,0.000000e+00,100000000
...,...,...,...,...,...
QWC,Qwertycoin,CryptoNight Heavy,PoW,9.955311e+10,184470000000
NYC,NewYorkCoin,Scrypt,PoW,1.430067e+11,0
GCN,gCn Coin,Scrypt,PoW,1.630551e+11,200000000000
BCN,ByteCoin,CryptoNight,PoW,1.840668e+11,184467440735


In [153]:
# Keep the rows where coins are mined.
crypto_df = crypto_df.loc[crypto_df["TotalCoinsMined"] > 0]

In [154]:
#crypto_df = crypto_df.drop(crypto_df[crypto_df['TotalCoinsMined'] > 0].index)

In [155]:
crypto_df.shape

(532, 5)

In [156]:
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [157]:
# Create a new DataFrame that holds only the cryptocurrencies names.
col = ["CoinName"]
crypto_names_df = crypto_df[col]

In [158]:
type(crypto_names_df)

pandas.core.frame.DataFrame

In [159]:
crypto_names_df.shape

(532, 1)

In [160]:
crypto_names_df.head(10)

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
LTC,Litecoin
DASH,Dash
XMR,Monero
ETC,Ethereum Classic
ZEC,ZCash


In [161]:
# Drop the 'CoinName' column as it will not be used on the clustering algorithm
crypto_df = crypto_df.drop(columns="CoinName")

In [162]:
crypto_df.shape

(532, 4)

In [163]:
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [164]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])

In [165]:
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [166]:
# Standardize the data with StandardScaler()
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [167]:
X

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [168]:
# reduce the dimension to three principal components
pca = PCA(n_components=3)

pcs = pca.fit_transform(X)

In [169]:
pcs

array([[-0.3183268 ,  1.05927188, -0.49085928],
       [-0.30167061,  1.05986338, -0.4911834 ],
       [ 2.30157867,  1.77305494, -0.56088784],
       ...,
       [ 0.32050951, -2.29337552,  0.35716243],
       [-0.1690936 , -2.08110885,  0.29508204],
       [-0.27630002,  0.87401032, -0.24883119]])

In [170]:
pcs_df = pd.DataFrame(
    data=pcs,
    index=[crypto_df.index],
    columns=['PC1', 'PC2', 'PC3']
)

pcs_df.head(10)

,PC1,PC2,PC3
42,-0.318327,1.059272,-0.490859
404,-0.301671,1.059863,-0.491183
1337,2.301579,1.773055,-0.560888
BTC,-0.157872,-1.324861,0.143361
ETH,-0.160938,-2.047247,0.392644
LTC,-0.151280,-1.110417,-0.013167
DASH,-0.404964,1.238229,-0.399082
XMR,-0.158384,-2.217018,0.370364
ETC,-0.159380,-2.047301,0.392636
ZEC,-0.169093,-2.081109,0.295082


In [171]:
pcs_df.shape

(532, 3)

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [172]:
# Create an elbow curve to find the best value for K.
inertia = []

k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    
    km = KMeans(n_clusters=i, random_state=0)
    
    km.fit(pcs_df)
    
    inertia.append(km.inertia_)

In [173]:
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}

df_elbow = pd.DataFrame(elbow_data)

df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [174]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4)

model

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

predictions

array([0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2,
       2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0,
       2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2,
       2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 0,
       0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2,
       0, 2, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0,
       2, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0,
       0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0,
       0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2,
       0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 2, 0,

In [175]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Join the crypto_df and pcs_df DataFrames on the same columns.
# clustered_df = pd.concat([crypto_df, pcs_df], axis=0)
clustered_df = crypto_df.join(pcs_df)

# #  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = crypto_names_df["CoinName"]

# #  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,NaN,NaN,NaN,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,NaN,NaN,NaN,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,NaN,NaN,NaN,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,NaN,NaN,NaN,Bitcoin,2
ETH,Ethash,PoW,1.076842e+08,0,NaN,NaN,NaN,Ethereum,2
LTC,Scrypt,PoW,6.303924e+07,84000000,NaN,NaN,NaN,Litecoin,2
DASH,X11,PoW/PoS,9.031294e+06,22000000,NaN,NaN,NaN,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,NaN,NaN,NaN,Monero,2
ETC,Ethash,PoW,1.133597e+08,210000000,NaN,NaN,NaN,Ethereum Classic,2
ZEC,Equihash,PoW,7.383056e+06,21000000,NaN,NaN,NaN,ZCash,2


In [176]:
clustered_df.shape

(532, 9)

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [189]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name='CoinName',
    hover_data=['Algorithm']
)

fig.update_layout(legend=dict(x=0, y=1))

fig.show()

In [190]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [195]:
# Print the total number of tradable cryptocurrencies.
print(f'There are {len(clustered_df)} tradable cryptocurrencies.')

There are 532 tradable cryptocurrencies.


In [197]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaled_df = MinMaxScaler().fit_transform(clustered_df[['TotalCoinSupply', 'TotalCoinsMined']])

In [198]:
scaled_df

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [205]:
scaled_df.shape

(532, 2)

In [207]:
type(scaled_df)

numpy.ndarray

In [210]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(
    scaled_df,
    columns=['TotalCoinSupply', 'TotalCoinsMined'],
    index=clustered_df.index
)

In [212]:
plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined
42,4.200000e-11,0.000000
404,5.320000e-04,0.001066
1337,3.141593e-01,0.029576
BTC,2.100000e-05,0.000018
ETH,0.000000e+00,0.000109
LTC,8.400000e-05,0.000064
DASH,2.200000e-05,0.000009
XMR,0.000000e+00,0.000017
ETC,2.100000e-04,0.000115
ZEC,2.100000e-05,0.000007


In [213]:
plot_df["CoinName"] = clustered_df["CoinName"]

In [215]:
plot_df["Class"] = clustered_df["Class"]

In [216]:
plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,2
ETH,0.000000e+00,0.000109,Ethereum,2
LTC,8.400000e-05,0.000064,Litecoin,2
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,2
ETC,2.100000e-04,0.000115,Ethereum Classic,2
ZEC,2.100000e-05,0.000007,ZCash,2


In [218]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class", hover_cols="CoinName")

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)